In [26]:
from bs4 import BeautifulSoup
import re
import requests
import operator
import threading
from threading import Thread
import pickle
from datetime import datetime

In [2]:
file_base = %pwd
file_base += "/cve-data/"
file_base

'/Users/bala/Desktop/227/code/CSE-227/cve-data/'

In [3]:
CVE_IDS = []
file = "download ("
for i in range(0,29):
    file_path = file_base + file + str(i) + ").tsv"
    with open(file_path) as f:
        contents = f.readlines()
        for line in contents[1:]:
            words = line.split('\t')
            CVE_IDS.append(words[1])

In [11]:
CVE_TO_BUGID = dict()

for cve in CVE_IDS:
    url = "http://www.cvedetails.com/cve/" + cve + "/"
    hdr = {'User-Agent': 'Mozilla/5.0'}
    html = requests.get(url)
    soup = BeautifulSoup(html.text, "html.parser")
    
    table = soup.find("table", {"id": "vulnrefstable"})
    a_tags = table.find_all('a')
    for a in a_tags:
        if "bugzilla" in a['href']:
            if cve in CVE_TO_BUGID:
                CVE_TO_BUGID[cve].append(a['href'])
            else:
                CVE_TO_BUGID[cve] = [a['href']]

# some CVE entries don't have bugzilla links
print(len(CVE_TO_BUGID))

1233


In [14]:
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [16]:
#storing the dict
save_obj(CVE_TO_BUGID, "cve_to_bugzilla_links")

In [30]:
# cve to earliest modified date on bugzilla
# component frequency in security vulnerabilities
CVE_TO_DATE = dict()
COMPONENT_FREQUENCY = dict()

for key_cve, value_links in CVE_TO_BUGID.items():
    print(key_cve)
    main_date = None
    for l in value_links:
        s_url = l
        s_hdr = {'User-Agent': 'Mozilla/5.0'}
        s_html = requests.get(s_url)
        s_soup = BeautifulSoup(s_html.text, "html.parser")
        
        startDate = s_soup.find("span", attrs={"id": "field-value-creation_ts"})
        if startDate != None:
            startDate = startDate.find("span", attrs={"class": "rel-time"})["title"].strip()
            temp = datetime.strptime(startDate, '%Y-%m-%d %H:%M %Z')
            if main_date == None:
                main_date = temp
            else:
                if temp < main_date:
                    main_date = temp
        
        comp = s_soup.find("div", attrs={"id": "component-name"})
        if comp != None:
            t_comp = comp.text.strip().split(":")[0]
            if t_comp in COMPONENT_FREQUENCY:
                COMPONENT_FREQUENCY[t_comp] += 1
            else:
                COMPONENT_FREQUENCY[t_comp] = 1

    if main_date != None:
        CVE_TO_DATE[key_cve] = main_date
        

CVE-2016-5284
CVE-2016-5283
CVE-2016-5282
CVE-2016-5281
CVE-2016-5280
CVE-2016-5279
CVE-2016-5278
CVE-2016-5277
CVE-2016-5276
CVE-2016-5275
CVE-2016-5274
CVE-2016-5273
CVE-2016-5272
CVE-2016-5271
CVE-2016-5270
CVE-2016-5268
CVE-2016-5267
CVE-2016-5266
CVE-2016-5265
CVE-2016-5264
CVE-2016-5263
CVE-2016-5262
CVE-2016-5261
CVE-2016-5260
CVE-2016-5259
CVE-2016-5258
CVE-2016-5257
CVE-2016-5256
CVE-2016-5255
CVE-2016-5254
CVE-2016-5253
CVE-2016-5252
CVE-2016-5251
CVE-2016-5250
CVE-2016-2839
CVE-2016-2838
CVE-2016-2837
CVE-2016-2836
CVE-2016-2835
CVE-2016-2834
CVE-2016-2833
CVE-2016-2832
CVE-2016-2831
CVE-2016-2830
CVE-2016-2829
CVE-2016-2828
CVE-2016-2827
CVE-2016-2826
CVE-2016-2825
CVE-2016-2824
CVE-2016-2822
CVE-2016-2821
CVE-2016-2820
CVE-2016-2819
CVE-2016-2818
CVE-2016-2817
CVE-2016-2816
CVE-2016-2815
CVE-2016-2814
CVE-2016-2813
CVE-2016-2812
CVE-2016-2811
CVE-2016-2810
CVE-2016-2809
CVE-2016-2808
CVE-2016-2807
CVE-2016-2806
CVE-2016-2804
CVE-2016-2802
CVE-2016-2801
CVE-2016-2800
CVE-20

CVE-2012-4207
CVE-2012-4206
CVE-2012-4205
CVE-2012-4204
CVE-2012-4203
CVE-2012-4202
CVE-2012-4201
CVE-2012-4196
CVE-2012-4195
CVE-2012-4194
CVE-2012-4193
CVE-2012-4192
CVE-2012-4191
CVE-2012-4190
CVE-2012-4188
CVE-2012-4187
CVE-2012-4186
CVE-2012-4185
CVE-2012-4184
CVE-2012-4183
CVE-2012-4182
CVE-2012-4181
CVE-2012-4180
CVE-2012-4179
CVE-2012-3995
CVE-2012-3994
CVE-2012-3993
CVE-2012-3992
CVE-2012-3991
CVE-2012-3990
CVE-2012-3989
CVE-2012-3988
CVE-2012-3987
CVE-2012-3986
CVE-2012-3985
CVE-2012-3984
CVE-2012-3983
CVE-2012-3982
CVE-2012-3980
CVE-2012-3979
CVE-2012-3978
CVE-2012-3976
CVE-2012-3975
CVE-2012-3974
CVE-2012-3973
CVE-2012-3972
CVE-2012-3971
CVE-2012-3970
CVE-2012-3969
CVE-2012-3968
CVE-2012-3967
CVE-2012-3966
CVE-2012-3965
CVE-2012-3964
CVE-2012-3963
CVE-2012-3962
CVE-2012-3961
CVE-2012-3960
CVE-2012-3959
CVE-2012-3958
CVE-2012-3957
CVE-2012-3956
CVE-2012-3105
CVE-2012-1976
CVE-2012-1975
CVE-2012-1974
CVE-2012-1973
CVE-2012-1972
CVE-2012-1971
CVE-2012-1970
CVE-2012-1967
CVE-20

CVE-2005-2705
CVE-2005-2395
CVE-2005-2270
CVE-2005-2269
CVE-2005-2268
CVE-2005-2267
CVE-2005-2266
CVE-2005-2265
CVE-2005-2264
CVE-2005-2263
CVE-2005-2261
CVE-2005-2260
CVE-2005-1937
CVE-2005-1477
CVE-2005-1476
CVE-2005-1160
CVE-2005-1159
CVE-2005-1158
CVE-2005-1157
CVE-2005-1156
CVE-2005-1155
CVE-2005-1154
CVE-2005-1153
CVE-2005-0989
CVE-2005-0593
CVE-2005-0592
CVE-2005-0591
CVE-2005-0590
CVE-2005-0589
CVE-2005-0588
CVE-2005-0584
CVE-2005-0402
CVE-2005-0399
CVE-2005-0238
CVE-2005-0232
CVE-2005-0231
CVE-2005-0230
CVE-2005-0150
CVE-2005-0147
CVE-2005-0146
CVE-2005-0145
CVE-2005-0144
CVE-2005-0143
CVE-2005-0142
CVE-2005-0141
CVE-2004-2657
CVE-2004-2227
CVE-2004-2225
CVE-2004-1753
CVE-2004-0905
CVE-2004-0904
CVE-2004-0867
CVE-2004-0779
CVE-2004-0765
CVE-2004-0764
CVE-2004-0763
CVE-2004-0762
CVE-2004-0761
CVE-2004-0757
CVE-2002-2437
CVE-2002-2436


In [32]:
#storing the dict of cve -> date from bugzilla
save_obj(CVE_TO_DATE, "cve_to_bugzilla_date")